In [1]:
# import the necessary packages
import numpy as np
from imutils import face_utils
import dlib
import cv2
import math
import imutils
import face_recognition

from locations import landmarks

In [2]:
# initializing dlib's face detector and encoder, creating predictor
p = "../data/shape_predictor_194_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [3]:
def calc_angle(v1, v2):
    '''
    计算两个向量的夹角：
    AB = [5,-1,1,-3]
    CD = [4,1,4.5,4.5]
    print(angle_new(AB, CD),type(AB))
    '''
    dx1 = v1[2] - v1[0]
    dy1 = v1[3] - v1[1]
    dx2 = v2[2] - v2[0]
    dy2 = v2[3] - v2[1]
    angle1 = math.atan2(dy1, dx1)
    angle1 = int(angle1 * 180/math.pi)
    # print(angle1)
    angle2 = math.atan2(dy2, dx2)
    angle2 = int(angle2 * 180/math.pi)
    # print(angle2)
    if angle1*angle2 >= 0:
        included_angle = abs(angle1-angle2)
    else:
        included_angle = abs(angle1) + abs(angle2)
        if included_angle > 180:
            included_angle = 360 - included_angle
    return included_angle

def calc_wuguan_std(center_list):
    """
    输入一张脸在一个视频中的的五官中心点列表
    """
    if len(center_list) == 1:
        return np.array([0])
    elif len(center_list) > 1:
        return np.array([np.linalg.norm(np.array(center_list[i])-np.array(center_list[i+1]),axis=1).std() \
                for i in range(len(center_list)-1)])

In [4]:
# calc features
def feature_calc(video_path, n_frames=30):
    # Create video reader and find length
    v_cap = cv2.VideoCapture(video_path)
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # pick n frames
    sample = np.linspace(0, v_len - 1, n_frames).astype(int)

    # by frame index
    face_color_avgs = []
    face_color_stds = []
    yanpi_face_avgs = []
    lip_color_avgs = []
    face_fluctuation = []
    brow_dists = []
    brow2nose_dists = []
    brow2eye_dists = []
    brow2nose_angles = []
    brow_thicks = []
    face_rects = []
    u_lip_thickness = []
    d_lip_thickness = []
    u_lip_width = []
    d_lip_width = []
    l_brow_length = []
    r_brow_length = []
    nose_width = []
    l_eye_width = []
    r_eye_width = []
    l_eye_height = []
    r_eye_height = []
    nose2ulip_dists = []
    xiaba2dlip_dists = []
    nose_color_avgs = []
    
    
    # part counter indexes
    r_eye_index = landmarks().get_polygons_index('R_EYE')
    l_eye_index = landmarks().get_polygons_index('L_EYE')
    xiaba_index = landmarks().get_polygons_index('XIABA')
    l_brow_index = landmarks().get_polygons_index('L_BROW')
    r_brow_index = landmarks().get_polygons_index('R_BROW')
    nose_index = landmarks().get_polygons_index('NOSE')
    u_lip_index = landmarks().get_polygons_index('U_LIP')
    d_lip_index = landmarks().get_polygons_index('D_LIP')
    
    latest_encodings = [] # init a list of the lastest encodings of each cluster of faces
    flag = 0 # set a detector flag in one video while looping
    
    for j in range(v_len):
        success = v_cap.grab()
        if j in sample:
#             print('=========第{}帧========'.format(j))
            success, frame = v_cap.retrieve()
            if not success:
                continue
            frame = imutils.resize(frame, width = int(frame.shape[1]/1.5))
            
            # detect faces in the grayscale image
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            rects = detector(gray, 0)
            
            # get encodings list of faces
            known_face_locations = [(rect.top(), rect.right(), rect.bottom(), rect.left()) for rect in rects]
            biden_encoding = face_recognition.face_encodings(face_image=frame, 
                                                             known_face_locations=known_face_locations)
                        
            for (i, rect) in enumerate(rects):
                f_encoding = biden_encoding[i] # the encoding of i-th face
                
                # find the face index of this rect update latest_encodings list
                if latest_encodings:
                    results = face_recognition.compare_faces(latest_encodings, f_encoding)
                    if sum(results) == 0:
                        flag = 1
                    else:
                        true_index = results.index(True)
                        latest_encodings[true_index] = f_encoding
                else:
                    flag = 1
                
                if flag:
                    latest_encodings.append(f_encoding)
                    true_index = len(latest_encodings) - 1
                    face_color_avgs.append([])
                    face_color_stds.append([])
                    yanpi_face_avgs.append([])
                    lip_color_avgs.append([])
                    face_fluctuation.append([])
                    brow_dists.append([])
                    brow2nose_dists.append([])
                    brow2eye_dists.append([])
                    brow2nose_angles.append([[],[]])
                    brow_thicks.append([])
                    face_rects.append([])
                    u_lip_thickness.append([])
                    d_lip_thickness.append([])
                    u_lip_width.append([])
                    d_lip_width.append([])
                    l_brow_length.append([])
                    r_brow_length.append([])
                    nose_width.append([])
                    l_eye_width.append([])
                    r_eye_width.append([])
                    l_eye_height.append([])
                    r_eye_height.append([])
                    nose2ulip_dists.append([])
                    xiaba2dlip_dists.append([])
                    nose_color_avgs.append([])
                    
                    flag = 0
                
                shape = predictor(gray, rect)
                shape = face_utils.shape_to_np(shape)
                    
                xia_lian = abs(shape[0][1] - shape[109][1])
                to_append_pnt = [[shape[0][0], int(shape[0][1] + xia_lian * 0.4)], 
                                   [shape[134][0], int(shape[134][1] + xia_lian * 0.4)]]

                # part counter points
                l_eye = shape[l_eye_index]
                r_eye = shape[r_eye_index]
                xiaba = np.concatenate((shape[xiaba_index], to_append_pnt), axis=0)
                u_lip = shape[u_lip_index]
                d_lip = shape[d_lip_index]
                l_brow = shape[l_brow_index]
                r_brow = shape[r_brow_index]
                nose = shape[nose_index]

                # make masks with coordinates
                mask_xiaba = np.zeros((frame.shape[0], frame.shape[1]))
                mask_eye = np.zeros((frame.shape[0], frame.shape[1]))
                mask_lip = np.zeros((frame.shape[0], frame.shape[1]))
                mask_nose = np.zeros((frame.shape[0], frame.shape[1]))

                cv2.fillPoly(mask_xiaba, [xiaba], 1)
                cv2.fillPoly(mask_eye, [l_eye, r_eye], 1)
                cv2.fillPoly(mask_lip, [u_lip, d_lip], 1)
                cv2.fillPoly(mask_nose, [nose], 1)

                mask_xiaba = mask_xiaba.astype(np.bool)
                xiaba_out = np.zeros_like(gray)
                xiaba_out[mask_xiaba] = gray[mask_xiaba]
                mask_eye = mask_eye.astype(np.bool)
                eye_out = np.zeros_like(gray)
                eye_out[mask_eye] = gray[mask_eye]
                mask_lip = mask_lip.astype(np.bool)
                lip_out = np.zeros_like(gray)
                lip_out[mask_lip] = gray[mask_lip]
                mask_nose = mask_nose.astype(np.bool)
                nose_out = np.zeros_like(gray)
                nose_out[mask_nose] = gray[mask_nose]

                l_x, l_y, l_w, l_h = cv2.boundingRect(l_eye)
                l_eye_rectangle = gray[int(l_y-0.25*l_h):l_y+l_h, l_x:l_x+l_w]
                r_x, r_y, r_w, r_h = cv2.boundingRect(r_eye)
                r_eye_rectangle = gray[int(r_y-0.25*r_h):r_y+r_h, r_x:r_x+r_w]

                # distances
                center = [l_brow.mean(axis=0).tolist(),
                          r_brow.mean(axis=0).tolist(),
                          l_eye.mean(axis=0).tolist(),
                          r_eye.mean(axis=0).tolist(),
                          nose.mean(axis=0).tolist(),
                          np.concatenate((u_lip,d_lip)).mean(axis=0).tolist()]
                brow_dist = np.linalg.norm(shape[70] - shape[82])
                brow2nose_dist = np.linalg.norm(np.concatenate((shape[70],shape[82])).mean(axis=0) - shape[143])
                brow2eye_dist = (np.linalg.norm(np.array(center[0])-np.array(center[2])) + np.linalg.norm(np.array(center[1])-np.array(center[3]))) / 2
                brow_thick = (np.linalg.norm(shape[96] - shape[110]) + np.linalg.norm(shape[74] - shape[88])) / 2
                u_lip_dist = np.linalg.norm(shape[159] - shape[187])
                d_lip_dist = np.linalg.norm(shape[19] - shape[174])
                u_lip_width_dist = np.linalg.norm(shape[152] - shape[165])
                d_lip_width_dist = np.linalg.norm(shape[25] - shape[11])
                l_brow_len_dist = np.linalg.norm(shape[103] - shape[92])
                r_brow_len_dist = np.linalg.norm(shape[70] - shape[81])
                nose_width_dist = np.linalg.norm(shape[139] - shape[147])
                l_eye_width_dist = np.linalg.norm(shape[59] - shape[48])
                r_eye_width_dist = np.linalg.norm(shape[26] - shape[38])
                l_eye_height_dist = np.linalg.norm(shape[55] - shape[64])
                r_eye_height_dist = np.linalg.norm(shape[31] - shape[44])
                nose2ulip_dist = np.linalg.norm(nose.mean(axis=0) - u_lip.mean(axis=0))
                xiaba2dlip_dist = np.linalg.norm(shape[xiaba_index].mean(axis=0) - d_lip.mean(axis=0))

                # angles
                nose_vec = np.array([shape[139], shape[147]]).mean(axis=0).tolist() + np.array([shape[135], shape[151]]).mean(axis=0).tolist()
                l_brow_vec = shape[103].tolist() + shape[113].tolist()
                r_brow_vec = shape[81].tolist() + shape[91].tolist()
                l_brow2nose_angle = calc_angle(nose_vec, l_brow_vec)
                r_brow2nose_angle = calc_angle(nose_vec, r_brow_vec)


                # calc feature and drop them into corresponding face index
                face_color_avg = xiaba_out.sum() / mask_xiaba.sum()
                face_color_avgs[true_index].append(face_color_avg)
                face_color_stds[true_index].append(xiaba_out.std())
                yanpi_avg = (l_eye_rectangle.sum() + r_eye_rectangle.sum() - eye_out.sum()) / ((l_eye_rectangle.shape[0] + \
                        r_eye_rectangle.shape[0]) * (l_eye_rectangle.shape[1] + r_eye_rectangle.shape[1]) - mask_eye.sum())
                yanpi_face_avgs[true_index].append(yanpi_avg)
                lip_color_avg = lip_out.sum() / mask_lip.sum()
                lip_color_avgs[true_index].append(lip_color_avg)
                face_fluctuation[true_index].append(center)
                brow_dists[true_index].append(brow_dist)
                brow2nose_dists[true_index].append(brow2nose_dist)
                brow2eye_dists[true_index].append(brow2eye_dist)
                brow2nose_angles[true_index][0].append(l_brow2nose_angle)
                brow2nose_angles[true_index][1].append(r_brow2nose_angle)
                brow_thicks[true_index].append(brow_thick)
                u_lip_thickness[true_index].append(u_lip_dist)
                d_lip_thickness[true_index].append(d_lip_dist)
                u_lip_width[true_index].append(u_lip_width_dist)
                d_lip_width[true_index].append(d_lip_width_dist)
                l_brow_length[true_index].append(l_brow_len_dist)
                r_brow_length[true_index].append(r_brow_len_dist)
                nose_width[true_index].append(nose_width_dist)
                l_eye_width[true_index].append(l_eye_width_dist)
                r_eye_width[true_index].append(r_eye_width_dist)
                l_eye_height[true_index].append(l_eye_height_dist)
                r_eye_height[true_index].append(r_eye_height_dist)
                nose2ulip_dists[true_index].append(nose2ulip_dist)
                xiaba2dlip_dists[true_index].append(xiaba2dlip_dist)
                nose_color_avg = nose_out.sum() / mask_nose.sum()
                nose_color_avgs[true_index].append(nose_color_avg)
                face_rects[true_index].append(rect.left())

    v_cap.release() # release video
    
    face_cnt = len(face_color_avgs) # 脸的数量
    feature_face_color = [[np.array(face_color_avgs[i]).std()] for i in range(face_cnt)] # 面部色度标准差
    feature_face_maxdiff = [[max(face_color_stds[i])] for i in range(face_cnt)] # 面部色度最大差异（最大标准差）
    feature_yanpi_face_diff = [[np.array(yanpi_face_avgs[i]).std()] for i in range(face_cnt)] # 眼皮面部色度差的标准差
    feature_lip_color = [[np.array(lip_color_avgs[i]).std()] for i in range(face_cnt)] # 嘴唇色度标准差
    feature_face_fluctuation = [[max(calc_wuguan_std(face_fluctuation[i]))] for i in range(face_cnt)] # 面部特征移动相对距离
    feature_brow_dist = [[np.array(brow_dists[i]).std()] for i in range(face_cnt)] # 眉毛距离标准差
    feature_brow2nose_dist = [[np.array(brow2nose_dists[i]).std()] for i in range(face_cnt)] # 眉心到鼻尖距离标准差
    feature_brow2eye_dist = [[np.array(brow2eye_dists[i]).std()] for i in range(face_cnt)] # 眉毛到眼睛距离标准差
    feature_brow2nose_angle = [[np.array(brow2nose_angles[i][0]).std(),np.array(brow2nose_angles[i][1]).std()] 
                               for i in range(face_cnt)] # 眉毛与鼻子夹角，左右眉毛都做了计算
    feature_brow_thick = [[np.array(brow_thicks[i]).std()] for i in range(face_cnt)] # 眉毛粗细
    feature_ulip_thickness = [[np.array(u_lip_thickness[i]).std()] for i in range(face_cnt)] # 上嘴唇厚度标准差
    feature_dlip_thickness = [[np.array(d_lip_thickness[i]).std()] for i in range(face_cnt)] # 下嘴唇厚度标准差
    feature_ulip_width = [[np.array(u_lip_width[i]).std()] for i in range(face_cnt)] # 上嘴唇宽度标准差
    feature_dlip_width = [[np.array(d_lip_width[i]).std()] for i in range(face_cnt)] # 下嘴唇宽度标准差
    feature_lbrow_len = [[np.array(l_brow_length[i]).std()] for i in range(face_cnt)] # 上眉毛长度标准差
    feature_rbrow_len = [[np.array(r_brow_length[i]).std()] for i in range(face_cnt)] # 下眉毛长度标准差
    feature_nose_width = [[np.array(nose_width[i]).std()] for i in range(face_cnt)] # 鼻子宽度标准差
    feature_leye_width = [[np.array(l_eye_width[i]).std()] for i in range(face_cnt)] # 左眼宽度标准差
    feature_reye_width = [[np.array(r_eye_width[i]).std()] for i in range(face_cnt)] # 右眼宽度标准差
    feature_leye_height = [[np.array(l_eye_height[i]).std()] for i in range(face_cnt)] # 左眼高度标准差
    feature_reye_height = [[np.array(r_eye_height[i]).std()] for i in range(face_cnt)] # 右眼高度标准差
    feature_nose2ulip_dist = [[np.array(nose2ulip_dists[i]).std()] for i in range(face_cnt)] # 鼻子中心到上唇中心的距离标准差
    feature_xiaba2dlip_dist = [[np.array(xiaba2dlip_dists[i]).std()] for i in range(face_cnt)] # 下唇中心到下颚中心的距离标准差
    feature_nose_color = [[np.array(nose_color_avgs[i]).std()] for i in range(face_cnt)] # 鼻子区域的颜色标准差
    
    rect_left = [[np.array(face_rects[i]).mean()] for i in range(face_cnt)] # 计算每张脸与左边框的距离，以此来打标签

    feature = []
    for i in range(face_cnt):
        feature.append(feature_face_color[i] +
                       feature_face_maxdiff[i] +
                       feature_yanpi_face_diff[i] +
                       feature_lip_color[i] +
                       feature_face_fluctuation[i] + 
                       feature_brow_dist[i] + 
                       feature_brow2nose_dist[i] +
                       feature_brow2eye_dist[i] +
                       feature_brow2nose_angle[i] +
                       feature_brow_thick[i] + 
                       feature_ulip_thickness[i] + 
                       feature_dlip_thickness[i] + 
                       feature_ulip_width[i] + 
                       feature_dlip_width[i] + 
                       feature_lbrow_len[i] + 
                       feature_rbrow_len[i] + 
                       feature_nose_width[i] + 
                       feature_leye_width[i] + 
                       feature_reye_width[i] + 
                       feature_leye_height[i] + 
                       feature_reye_height[i] + 
                       feature_nose2ulip_dist[i] + 
                       feature_xiaba2dlip_dist[i] + 
                       feature_nose_color[i] + 
                       rect_left[i])
    print(feature)
    return feature

In [5]:
import pandas as pd
import json
import os
import time

def calc_folder(folder_num):
    cnt = 0
    out_path = "/home/sufedc_nvidia_handianyouhua/deep_fake/results/"
    data_path = "/home/sufedc_nvidia_handianyouhua/deep_fake/data/data/"
    metadata = json.loads(open(os.path.join(data_path, 'dfdc_train_part_{}'.format(folder_num), 'metadata.json')).read())
    features = []
    t1 = time.time()

    for video in metadata.keys():
        if cnt % 20 == 0:
            print('============{} FINISHED, {}min used========='.format(cnt, (time.time()-t1) / 60.0))   
        cnt += 1
        video_path = os.path.join(data_path, 'dfdc_train_part_{}'.format(folder_num), video)
        print(video)
        if os.path.exists(video_path):
            if metadata[video]['label'] == 'FAKE':
                label = 0
            else:
                label = 1
            feature = feature_calc(video_path, n_frames=20)
            if feature != []:
                for i in range(len(feature)):
                    features.append([video] + feature[i] + [label])

    res = pd.DataFrame(features,columns = ['video',
                                           'face_color',
                                           'face_maxdiff',
                                           'yanpi_face_diff',
                                           'lip_color',
                                           'face_fluctuation',
                                           'brow_dist',
                                           'brow2nose_dist',
                                           'brow2eye_dist',
                                           'l_brow2nose_angle',
                                           'r_brow2nose_angle',
                                           'brow_thick',
                                           'ulip_thickness',
                                           'dlip_thickness',
                                           'ulip_width',
                                           'dlip_width',
                                           'lbrow_len',
                                           'rbrow_len',
                                           'nose_width',
                                           'leye_width',
                                           'reye_width',
                                           'leye_height',
                                           'reye_height',
                                           'nose2ulip_dist',
                                           'xiaba2dlip_dist',
                                           'nose_color',
                                           'rect_left',
                                           'label'])

    res.to_csv(out_path+'dfdc_train_part_{}.csv'.format(folder_num), sep=',', index=False, header=True)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
calc_folder(46)

============0 FINISHED, 4.3710072835286455e-08min used=========
umkxoagtyp.mp4
[[10.783250467397249, 4.029727215616586, 3.8760475891192026, 17.917079551953716, 6.761616196765762, 5.466813176688682, 32.323714869472646, 0.9964643238324589, 5.026153819992205, 8.201354901613648, 0.7825817373348938, 0.42409311058705135, 0.6187490201578401, 4.355491319574699, 3.7926323237813486, 2.8929068963070326, 5.37618610714953, 2.024221849664984, 1.1634435177103837, 1.8828936061802541, 0.5766932826145281, 0.3823490907250719, 1.4185816275584968, 2.9781424302700907, 10.438593358429843, 592.4666666666667]]
icctbkjedq.mp4
[[2.0026600279997497, 2.3180627067142727, 1.9234702955682437, 3.1131308673605522, 3.2048784177848426, 2.5055938945725402, 9.181264366375155, 0.8337174039269254, 4.116734142496938, 4.5287967496897, 0.28929013614885657, 0.5075048748661513, 0.4346469716330496, 1.6283194633359266, 1.278130695805031, 1.012412713072764, 2.9064200076377005, 0.9585873020367444, 0.5373831421233533, 1.14208719027789

[[2.2355389689013165, 8.09760129063257, 2.983429616677814, 6.599107603250344, 6.0612988669249015, 2.763591044570056, 16.501876239339957, 1.3926104103305141, 3.335790760824186, 3.809199390948182, 0.7750090188227822, 0.6390362144046993, 0.6855606526837895, 2.7599049564747165, 2.094417197610113, 1.9936012459541497, 2.9421819637490967, 1.2434905712170579, 0.9324553690675904, 0.9990681659262425, 0.9407658823484556, 0.7263920333643956, 1.0967984725869855, 3.9331611345447146, 5.714591433553451, 656.95]]
clzcvdsuyp.mp4
[[2.1548392352253107, 2.428364459430558, 3.010809537192065, 2.6020597219782067, 4.554768065874887, 2.3314914319241664, 21.432505093400884, 0.5284478289767495, 2.8508770580296865, 3.6687872655688283, 0.33895180985617024, 0.765197669817223, 0.47332391613771485, 1.5758460346625185, 1.488667584377564, 2.5858306244406712, 2.5952903711006354, 0.8042330580725159, 0.7795644059284034, 0.9197503012509496, 0.5110945330318857, 0.46926269717604546, 0.4765485037446184, 2.154845854693725, 3.89

[[1.8947841150116802, 2.810151938103189, 2.196218334845641, 5.957394977134565, 3.6638510648505918, 1.8163126768135216, 10.716012518136335, 0.791325299654866, 5.59431844749356, 4.559243753201624, 0.5075517073299994, 0.34565165241599166, 0.4750393042100843, 2.186480666820895, 1.9632432754353808, 0.9299276849133848, 1.8920851565993848, 1.029836689377486, 0.574393498400384, 0.7318643028803333, 0.5438497046487714, 0.5311860376608841, 0.6196852621636971, 1.5755311854389922, 3.9438529488452216, 623.9473684210526]]
pmywbdljmw.mp4
[[1.8513702922948252, 2.682887530754845, 2.2542711865176472, 5.340293529840158, 3.9435362048067617, 1.8749162423322487, 10.418216267210804, 0.9056145645146795, 3.621266823530436, 5.36635667222353, 0.5744966711990002, 0.44016015363475536, 0.4952353835239034, 1.973753337269589, 1.8170814047431698, 0.716511298132568, 1.7955619744565212, 0.8235929425221798, 0.5731835285315975, 0.6312705107707388, 0.4866114944692901, 0.5204993710668139, 0.7041063452846914, 1.37280101422391

[[1.7660334211432533, 2.440331034062375, 2.0784458492235, 3.803046773047305, 3.441040300377819, 2.032596265131055, 12.155173557827087, 0.5201900092505481, 3.9749213828703582, 3.905124837953327, 0.495787593240089, 0.40113509985446555, 0.583630162522387, 1.6746282478096768, 1.406676695698162, 1.2624652809887205, 2.101893375810758, 0.5078671988834401, 0.5454187425244038, 0.6590707128886324, 0.5113830004564478, 0.565938437633769, 0.7171186368805909, 1.4042940958983476, 2.8337456283738547, 629.0]]
vbgkguxrvy.mp4
[[11.845547117479343, 3.404192674997215, 3.6757729771365204, 16.197008541690455, 4.220726894786956, 3.4432998839472453, 53.09296122213613, 1.0114389716196033, 4.256237253203298, 4.2405450383438, 0.7887816853598448, 0.47922288014738146, 0.6977856138299753, 4.064482617457388, 3.5472704161296376, 2.0932316829933413, 3.66876152837905, 1.3248304687185366, 0.48317218530899037, 1.332309330025047, 0.6748688918332353, 0.53270015309035, 1.0848585902891363, 2.6784327631088787, 16.4497982004807

[[1.2340090403296866, 3.166533716024629, 1.0851469306525166, 5.173868144133395, 3.190891528465599, 1.2102344648729124, 4.296168990037693, 0.7708984817298276, 1.8734993995195197, 1.4317821063276353, 0.6234004696939246, 0.3821852745255736, 0.5368984979217429, 2.6164093885301622, 2.5215716496483505, 0.75111905599334, 1.4854825424395748, 0.9322217981926989, 0.5608240586006469, 0.45784567129876214, 0.5273642683690841, 0.626625562234362, 1.3564090206877755, 1.1517338312512027, 3.135655259794932, 562.0]]
dwpoopdkmp.mp4
[[1.2643242300693567, 1.9039416861812843, 1.5252625058094083, 2.6063992825231463, 1.4085938439311807, 2.818404374387995, 7.114031634092797, 0.48802644349268637, 4.006914439041506, 5.543786072156901, 0.439489040800922, 0.46895159610847803, 0.4878532470089628, 1.3725524288159725, 1.2334063105380653, 0.6253962111408723, 2.6198529594926687, 1.0039053806049083, 0.5973898762011467, 0.6049326152144426, 0.4588619221718326, 0.42985987639232115, 0.22438650385585246, 1.8907621949621911, 2

...

[[1.7528953495880084, 2.4095595520732846, 2.10755626725922, 1.7716462842457323, 3.09761711641591, 2.343367965529836, 8.809239278721535, 0.8981864024262638, 3.7185346576306104, 3.9319206502674997, 0.24709967379623202, 0.4757478761256166, 0.5023983813017002, 1.4424322709989132, 1.3279046499509708, 0.6167515631377141, 2.113929778848932, 0.8372427292822597, 0.4975489730844551, 1.1378479318819448, 0.3855217835528314, 0.5999123448149067, 0.4283395718117759, 1.4608243767914986, 2.226883080323479, 646.2]]
thnywrwayf.mp4
[[4.3852074309106985, 8.714308538244085, 3.088850006449741, 8.22232856787395, 5.614995287811908, 2.91475965229126, 15.347976733185055, 2.6604463536037635, 2.576819745345025, 4.17612260356422, 0.6699009234144517, 0.6904193103634035, 1.1152836556210641, 4.078815966304048, 3.1959422942114752, 3.3701088748975367, 3.085978284602385, 1.2407992527264204, 1.5712511874185107, 1.5626280366523089, 1.6047480157765839, 1.3935000005815177, 1.4749846125084443, 3.7052303212869133, 7.5856624758

[[1.569851492979988, 2.6985585795985547, 2.982917513782082, 5.044271076989388, 3.3446807150327342, 1.809576416219331, 10.323459547714595, 0.7186568269638821, 3.7682152251934204, 3.581268597095579, 0.4153799659733924, 0.5442029037213751, 0.5337900653177235, 2.946555955551111, 2.666201064930445, 0.917166183466098, 1.814537680521151, 1.027405323721192, 0.5143399042500296, 0.6941445116975563, 0.59874992975268, 0.6124823464983346, 0.6502153553312472, 1.337514835653118, 3.300717731654395, 624.3684210526316]]
dcqdtsgrif.mp4
[[1.2966558290274361, 1.496213096022226, 1.6647436515054967, 2.8930120093470695, 3.6687703134875678, 1.310293921498384, 8.515898080467661, 1.263240537091228, 1.247219128924647, 5.354126134736337, 0.4607076007081735, 0.5798787266881973, 0.5439760265715496, 1.5609645780841095, 1.4314942006655753, 1.9358868188891698, 1.295382863447991, 0.9976003880005625, 0.9135277570998468, 0.8060212172062816, 0.4714045207910317, 0.4529386466152403, 1.2717921791377025, 0.27473662032328705, 4

[[1.2501566527743957, 3.980220989731605, 1.5652014501342766, 2.827968161346989, 5.558677304350529, 3.067231289419564, 19.07709218121446, 1.1915981849940132, 2.5, 1.710993863226867, 0.7083889432070472, 0.5610814247197161, 0.7318680124150372, 2.440412854372707, 2.136211071634246, 2.276764203238045, 3.2674404544145323, 0.9814793355384749, 0.8676538012436278, 1.2826236180018522, 0.9309333273249952, 0.8373690818755617, 1.0384493314335606, 2.1009850143397366, 2.6255929864615717, 615.1]]
rhhyonwjyg.mp4
[[1.9657993362959916, 1.5123692486351583, 1.9310111706531077, 4.655384151111838, 1.6822606270197804, 0.7485745805738973, 8.673335100844103, 0.9032945778439135, 2.8674417556808756, 3.681787005729087, 0.25655892711858247, 0.2792407799438736, 0.0, 0.8823519958060191, 1.3956503002357945, 0.7235078649414829, 0.4292202313811715, 0.49398854724210844, 0.7264571177233748, 0.45750484339616704, 0.9428090415820634, 0.7750175242971767, 0.7925025242504661, 0.5880310561741335, 0.693563200012134, 805.0]]
ftjkn

[[4.462476474716353, 7.166350064837049, 4.005170444771081, 4.6294201779653585, 9.389333909813192, 5.090111251936478, 18.479038650991793, 1.6446236042106497, 3.383415434143434, 4.78826690985371, 0.7659813095857694, 0.4496468621079121, 0.7659630395062437, 4.4659880791668884, 3.7928134180477526, 2.102860534416962, 5.342835848181389, 2.1599416396759206, 1.0710811064522914, 1.9859195213609806, 0.9958343879888599, 0.7231700780598698, 0.9256213413397743, 4.234933199607006, 5.731652114999606, 633.85]]
arzibochtu.mp4
[[1.818239786356822, 3.8536118368886885, 1.0615953826683329, 3.477718453102671, 3.5269838577328425, 2.6505637899526304, 5.373790249725929, 0.9798667064512723, 1.6462077633154326, 2.449489742783178, 0.7040575063033007, 0.41605071337279714, 0.7252556110892571, 1.492208511790847, 1.4902798197559215, 1.1149556478269689, 2.790832692041372, 1.1822000904153909, 0.9423830188786169, 1.1318567333366227, 0.6011001268456423, 0.48495914597086015, 0.6810648638830371, 0.5151244782287238, 3.694066

[[1.4903592768876512, 1.3898945384133792, 2.2269511320321453, 3.259467163781505, 0.6250382472773734, 0.4777132344448828, 2.604664710365334, 0.2471526518898685, 3.2998316455372216, 0.4714045207910317, 0.2569707829434336, 0.2792407799438736, 0.0, 1.2455609235807203, 0.8029417460112295, 0.0, 0.013084481749404239, 0.4637557475989666, 0.5748987704735122, 0.9428090415820634, 0.39490609816426686, 0.4714045207910317, 0.27281097872274485, 0.07423061408275676, 0.6913752937480067, 810.3333333333334]]
sngcdvisjk.mp4
[[12.112135712860383, 4.561839675676236, 4.389152362648465, 15.241362414808341, 5.8663944165502215, 2.4542015560553527, 26.89979679624619, 0.39210402984655235, 4.404038550643666, 6.270211763214665, 0.5043411541443141, 0.3921444367351272, 0.5800624328794561, 3.197153647510189, 2.855246052653024, 2.082771511061784, 2.6648722914480114, 1.7394454410940579, 1.1368029528354917, 0.9831722824109129, 0.6101289607913052, 0.6724374941456407, 0.9467151530922691, 2.0484066069574665, 13.500324910290